In [1]:
import numpy as np
from scipy.stats import multivariate_normal
from matplotlib import pyplot as plt
from copy import copy
from datetime import datetime, timedelta

from stonesoup.functions import gm_reduce_single

from stonesoup.predictor.kalman import KalmanPredictor
from stonesoup.updater.kalman import KalmanUpdater
from stonesoup.predictor.imm import IMMPredictor
from stonesoup.updater.imm import IMMUpdater
from stonesoup.models.transition.linear import ConstantVelocity, \
    CombinedLinearGaussianTransitionModel, LinearGaussianTimeInvariantTransitionModel, RandomWalk

from stonesoup.models.measurement.linear import LinearGaussian

from stonesoup.types.state import GaussianState
from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.detection import Detection
from stonesoup.types.state import StateVector, CovarianceMatrix, \
    GaussianMixtureState, WeightedGaussianState
from stonesoup.types.track import Track

from stonesoup.simulator.simple import SingleTargetGroundTruthSimulator

from matplotlib.patches import Ellipse


In [2]:
from emgpb2.states import Gaussian as EMGPB2Gaussian
from emgpb2.states import GMM as EMGPB2GMM
from emgpb2.states import GMMSequence as EMGPB2GMMSequence
from emgpb2.models import LinearModel as EMGPB2LinearModel
from emgpb2.models import ConstantVelocity as EMGPB2ConstantVelocity
from emgpb2.EM import SKFEstimator

Plot function

In [3]:
def plot_cov_ellipse(cov, pos, nstd=2, ax=None, **kwargs):
    """
    Plots an `nstd` sigma error ellipse based on the specified covariance
    matrix (`cov`). Additional keyword arguments are passed on to the
    ellipse patch artist.
    Parameters
    ----------
        cov : The 2x2 covariance matrix to base the ellipse on
        pos : The location of the center of the ellipse. Expects a 2-element
            sequence of [x0, y0].
        nstd : The radius of the ellipse in numbers of standard deviations.
            Defaults to 2 standard deviations.
        ax : The axis that the ellipse will be plotted on. Defaults to the
            current axis.
        Additional keyword arguments are pass on to the ellipse patch.
    Returns
    -------
        A matplotlib ellipse artist
    """

    def eigsorted(cov):
        vals, vecs = np.linalg.eigh(cov)
        order = vals.argsort()[::-1]
        return vals[order], vecs[:, order]

    if ax is None:
        ax = plt.gca()

    vals, vecs = eigsorted(cov)
    theta = np.degrees(np.arctan2(*vecs[:, 0][::-1]))

    # Width and height are "full" widths, not radius
    width, height = 2 * nstd * np.sqrt(vals)
    ellip = Ellipse(xy=pos, width=width, height=height, angle=theta,
                    **kwargs)

    ax.add_artist(ellip)
    return ellip


Groundtruth and detections

In [4]:
gt_track = []
gt_time = []
iter_model_1 = 120
iter_model_2 = 80

gt_transition_model_1 = CombinedLinearGaussianTransitionModel(
                        (RandomWalk(2.0 ** 2),
                        RandomWalk(2.0 ** 2))
                        )

timestamp_init = datetime.now()
state_init_1 = GaussianState(StateVector([[0], [0]]),
                           CovarianceMatrix(np.diag([0.0, 0.0])),
                           timestamp=timestamp_init)
gt_generator_1 = SingleTargetGroundTruthSimulator(gt_transition_model_1, state_init_1, number_steps=iter_model_1)

for time, gnd_paths in gt_generator_1.groundtruth_paths_gen():
    gnd_path = gnd_paths.pop()
    gt_track.append(gnd_path.state)
    gt_time.append(time)

gt_transition_model_2 = CombinedLinearGaussianTransitionModel(
                        (RandomWalk(10.0 ** 2),
                        RandomWalk(10.0 ** 2))
                        )
state_init_2 = GaussianState(gt_track[-1].state_vector,
                           CovarianceMatrix(np.diag([0.0, 0.0])),
                           timestamp=gt_time[-1])
gt_generator_2 = SingleTargetGroundTruthSimulator(gt_transition_model_2, state_init_2, number_steps=iter_model_2)
for time, gnd_paths in gt_generator_2.groundtruth_paths_gen():
    gnd_path = gnd_paths.pop()
    gt_track.append(gnd_path.state)
    gt_time.append(time)

detection_track = []
mesurement_noise_1 = np.diag([0.5 ** 2, 0.5 ** 2])
measurement_model_1 = LinearGaussian(ndim_state=2, mapping=[0, 1], noise_covar=mesurement_noise_1)
for i in range(iter_model_1):
    m_ = Detection(measurement_model_1.function(gt_track[i].state_vector, measurement_model_1.rvs(1)), gt_time[i])
    detection_track.append(m_)

mesurement_noise_2 = np.diag([0.75 ** 2, 0.75 ** 2])
measurement_model_2 = LinearGaussian(ndim_state=2, mapping=[0, 1], noise_covar=mesurement_noise_2)
for i in range(iter_model_2):
    ii = i + iter_model_1
    m_ = Detection(measurement_model_2.function(gt_track[ii].state_vector, measurement_model_2.rvs(1)), gt_time[ii])
    detection_track.append(m_)

# print(gt_track)
# print(detection_track)
# print(gt_time)

Filter model. Testing without EMGPB2 estimated parameters.

In [5]:
ft_transition_model_1 = CombinedLinearGaussianTransitionModel(
                        (RandomWalk(1.0 ** 2),
                        RandomWalk(1.0 ** 2))
                        )
ft_transition_model_2 = CombinedLinearGaussianTransitionModel(
                        (RandomWalk(20.0 ** 2),
                        RandomWalk(20.0 ** 2))
                        )
ft_measurement_noise = (2.0 **2) * np.eye(2)

ft_measurement_model = LinearGaussian(ndim_state=2, mapping=[0, 1],
                                   noise_covar=ft_measurement_noise)

ft_model_transition_matrix = np.array([[0.5, 0.5],
                                       [0.5, 0.5]])
ft_predictor_1 = KalmanPredictor(ft_transition_model_1)
ft_predictor_2 = KalmanPredictor(ft_transition_model_2)
ft_imm_predictor = IMMPredictor([ft_predictor_1, ft_predictor_2], ft_model_transition_matrix)

ft_updater = KalmanUpdater(ft_measurement_model)
ft_imm_updater = IMMUpdater([ft_updater, ft_updater], ft_model_transition_matrix)

ft_state_init = WeightedGaussianState(StateVector([[0], [0]]),
                                      CovarianceMatrix(np.diag([5.0, 5.0])),
                                      timestamp=timestamp_init,
                                      weight=0.5)

prior1 = copy(ft_state_init)
prior = GaussianMixtureState([prior1, prior1])
track = Track([copy(prior)])
track_error = []

for i in range(1, len(detection_track)):
    measurement = detection_track[i]
    # State prediction
    prediction = ft_imm_predictor.predict(track.state, timestamp=gt_time[i])
    # Measurement prediction
    meas_prediction = ft_imm_updater.predict_measurement(prediction)
    # State update
    hyp = SingleHypothesis(prediction, measurement)
    prior = ft_imm_updater.update(hyp)
    track.append(prior)
    track_error.append(ft_measurement_model.function(track[i].state_vector, np.zeros((2,1))) - ft_measurement_model.function(gt_track[i].state_vector, np.zeros((2,1))))
    '''
    fig, (ax1, ax2) = plt.subplots(2,1)
    # Generate plots
    ax1.cla()
    ax2.cla()
    # PLot true trajectory
    data = np.array([state.state_vector for state in gt_track])
    ax1.plot(data[:, 0], data[:, 2], 'b-')
    # PLot estimated trajectory
    data = np.array([state.state_vector for state in track.states])
    ax1.plot(data[:, 0], data[:, 2], 'r-')
    # Plot innovation covariance
    plot_cov_ellipse(meas_prediction.covar,
                     meas_prediction.mean, edgecolor='b',
                     facecolor='none', ax=ax1)
    # Plot estimated covariance
    plot_cov_ellipse(track.state.covar[[0, 2], :][:, [0, 2]],
                     track.state.mean[[0, 2], :], edgecolor='r',
                     facecolor='none', ax=ax1)
    # Visualise model weights
    ax2.bar([1,2], prior.weights.ravel())
    plt.pause(0.0001)
    '''

track_error = np.asarray(track_error)
track_error = np.squeeze(track_error)
rmse_tmp = []
for track_error_ in track_error:
    rmse_tmp.append(track_error_ @ track_error_.T)
rmse = np.sqrt(np.mean(np.array(rmse_tmp)))
print("RMSE: " + str(rmse))

RMSE: 1.6467973177639703


Filter model. Testing with correct parameters.

In [6]:
model_transition_matrix = np.array([[0.3, 0.7],
                                    [0.3, 0.7]])

predictor_1 = KalmanPredictor(gt_transition_model_1)
predictor_2 = KalmanPredictor(gt_transition_model_2)
imm_predictor = IMMPredictor([predictor_1, predictor_2],
                             model_transition_matrix)

updater_1 = KalmanUpdater(measurement_model_1)
updater_2 = KalmanUpdater(measurement_model_2)
imm_updater = IMMUpdater([updater_1, updater_2], model_transition_matrix)

prior1 = copy(ft_state_init)
prior = GaussianMixtureState([prior1, prior1])
track = Track([copy(prior)])
track_error = []

for i in range(1, len(detection_track)):
    measurement = detection_track[i]
    # State prediction
    prediction = imm_predictor.predict(track.state, timestamp=gt_time[i])
    # Measurement prediction
    meas_prediction = imm_updater.predict_measurement(prediction)
    # State update
    hyp = SingleHypothesis(prediction, measurement)
    prior = imm_updater.update(hyp)
    track.append(prior)
    track_error.append(ft_measurement_model.function(track[i].state_vector, np.zeros((2,1))) - ft_measurement_model.function(gt_track[i].state_vector, np.zeros((2,1))))
    # print(ft_measurement_model.function(track[i].state_vector, np.zeros((2,1))))
    # print(ft_measurement_model.function(gt_track[i].state_vector, np.zeros((2,1))))
    # print('----------------------')
    
    '''
    fig, (ax1, ax2) = plt.subplots(2,1)
    # Generate plots
    ax1.cla()
    ax2.cla()
    # PLot true trajectory
    data = np.array([state.state_vector for state in gt_track])
    ax1.plot(data[:, 0], data[:, 2], 'b-')
    # PLot estimated trajectory
    data = np.array([state.state_vector for state in track.states])
    ax1.plot(data[:, 0], data[:, 2], 'r-')
    # Plot innovation covariance
    plot_cov_ellipse(meas_prediction.covar,
                     meas_prediction.mean, edgecolor='b',
                     facecolor='none', ax=ax1)
    # Plot estimated covariance
    plot_cov_ellipse(track.state.covar[[0, 2], :][:, [0, 2]],
                     track.state.mean[[0, 2], :], edgecolor='r',
                     facecolor='none', ax=ax1)
    # Visualise model weights
    ax2.bar([1,2], prior.weights.ravel())
    plt.pause(0.0001)
    '''

track_error = np.asarray(track_error)
track_error = np.squeeze(track_error)
rmse_tmp = []
for track_error_ in track_error:
    rmse_tmp.append(track_error_ @ track_error_.T)
rmse = np.sqrt(np.mean(np.array(rmse_tmp)))
print("RMSE: " + str(rmse))

RMSE: 0.799715954243709


EMGPB2

In [7]:
F = np.asarray([
        [1, 0],
        [0, 1],
    ])
H = np.asanyarray([
    [1, 0],
    [0, 1]
])
def get_Q(Q_sig):
    Q = (Q_sig ** 2) * np.diag([1.0, 1.0])
    return Q
def transfer_Q(Q):
    new_Q = np.zeros_like(Q)
    for i in range(new_Q.shape[0]):
        new_Q[i, i] = Q[i, i]
    return new_Q

In [8]:
g1 = EMGPB2Gaussian(np.zeros([2, 1]), 10.0*np.eye(2))
g2 = EMGPB2Gaussian(np.zeros([2, 1]), 10.0*np.eye(2))
initial_gmm_state = EMGPB2GMM([g1, g2])

# measurements = 5 * np.random.randn(200, 2, 1) + 1

detection_track_EMGPB2 = [ele.state_vector for ele in detection_track]
gmmsequence = EMGPB2GMMSequence(np.float64(detection_track_EMGPB2), initial_gmm_state)

m1 = EMGPB2LinearModel(F, get_Q(1.0), H, (2.0 ** 2)*np.eye(2))
m2 = EMGPB2LinearModel(F, get_Q(20.0), H, (2.0 ** 2)*np.eye(2))
initial_models = [m1, m2]

Z = np.array([[0.5, 0.5], 
              [0.5, 0.5]])
dataset = [gmmsequence]

new_models_all, Z, dataset, LL = SKFEstimator.EM(dataset, initial_models, Z,
                                    max_iters=100, threshold=1e-6, learn_H=False, learn_R=True,
                                    learn_A=False, learn_Q=True, learn_init_state=False, learn_Z=True,
                                    diagonal_Q=False, wishart_prior=False)
new_models = new_models_all[-1]

new_models[0].Q = (new_models[0].Q + new_models[0].Q.T)/2
new_models[1].Q = (new_models[1].Q + new_models[1].Q.T)/2

-1282.701564864238
model -- 0 new_Q: 
[[3.01363645 0.01928178]
 [0.01798156 3.12719153]]
model -- 1 new_Q: 
[[131.12976587  -1.65058204]
 [ -1.6601313  135.08055238]]
model -- 0 new_R: 
[[3.29924997e+00 3.90306585e-04]
 [3.90306585e-04 2.94440853e+00]]
model -- 1 new_R: 
[[ 5.09981971 -0.10616243]
 [-0.10616243  5.00066726]]
new_Z: 
[[0.666579   0.32839588]
 [0.666579   0.32839588]]
-----------------------------------------------
-1231.6125658043038
model -- 0 new_Q: 
[[4.85955381 0.02167653]
 [0.03029722 4.70029594]]
model -- 1 new_Q: 
[[120.04618793   0.20193945]
 [  0.22911006 127.63495382]]
model -- 0 new_R: 
[[2.67471772e+00 1.84004364e-03]
 [1.85768496e-03 2.35118877e+00]]
model -- 1 new_R: 
[[ 5.66163784 -0.29317824]
 [-0.29308629  5.26198922]]
new_Z: 
[[0.67082961 0.32414527]
 [0.67082961 0.32414527]]
-----------------------------------------------
-1231.885325182221
model -- 0 new_Q: 
[[ 6.14327056e+00 -1.26649747e-02]
 [ 5.76137378e-03  5.62495953e+00]]
model -- 1 new_Q: 
[[1

-1235.5837142655832
model -- 0 new_Q: 
[[ 7.81798442 -1.00631949]
 [-0.98648574  6.44501598]]
model -- 1 new_Q: 
[[118.70884284   8.19726224]
 [  8.24337902 133.33358529]]
model -- 0 new_R: 
[[ 2.19426813 -0.63808687]
 [-0.63909924  1.64791053]]
model -- 1 new_R: 
[[ 7.17807993 -3.37545426]
 [-3.37847017  4.71712428]]
new_Z: 
[[0.69222239 0.30275248]
 [0.69222239 0.30275248]]
-----------------------------------------------
-1235.63854623205
model -- 0 new_Q: 
[[ 7.83053693 -1.0567426 ]
 [-1.03717499  6.45994573]]
model -- 1 new_Q: 
[[118.48236231   8.59467135]
 [  8.64061714 133.14840855]]
model -- 0 new_R: 
[[ 2.20717477 -0.6724844 ]
 [-0.67347827  1.65127445]]
model -- 1 new_R: 
[[ 7.27530831 -3.53935032]
 [-3.54229649  4.77620381]]
new_Z: 
[[0.69223148 0.30274339]
 [0.69223148 0.30274339]]
-----------------------------------------------
-1235.7110605922098
model -- 0 new_Q: 
[[ 7.8443581  -1.10723669]
 [-1.08794752  6.47720058]]
model -- 1 new_Q: 
[[118.2549079    8.99461993]
 [  9.

-1238.3127004214195
model -- 0 new_Q: 
[[ 8.10344591 -1.86557552]
 [-1.85235513  6.89379085]]
model -- 1 new_Q: 
[[114.48938812  15.2407199 ]
 [ 15.26494427 129.06661146]]
model -- 0 new_R: 
[[ 2.46199278 -1.28746676]
 [-1.28800702  1.87555768]]
model -- 1 new_R: 
[[ 9.02799996 -6.36348631]
 [-6.36498938  6.51501316]]
new_Z: 
[[0.69253727 0.3024376 ]
 [0.69253727 0.3024376 ]]
-----------------------------------------------
-1238.4306011423396
model -- 0 new_Q: 
[[ 8.107824   -1.89248286]
 [-1.87957704  6.9110997 ]]
model -- 1 new_Q: 
[[114.33829505  15.47288905]
 [ 15.49557827 128.85658334]]
model -- 0 new_R: 
[[ 2.46866317 -1.31198919]
 [-1.31250807  1.88866554]]
model -- 1 new_R: 
[[ 9.08098144 -6.4704476 ]
 [-6.47188895  6.6036382 ]]
new_Z: 
[[0.69244898 0.3025259 ]
 [0.69244898 0.3025259 ]]
-----------------------------------------------
-1238.5396313029064
model -- 0 new_Q: 
[[ 8.11086413 -1.91771405]
 [-1.90511247  6.92736355]]
model -- 1 new_Q: 
[[114.19544667  15.69135306]
 [ 1

-1239.3719167889974
model -- 0 new_Q: 
[[ 8.04278921 -2.1737024 ]
 [-2.16474083  7.09902713]]
model -- 1 new_Q: 
[[112.72174577  17.93969949]
 [ 17.94241758 125.70799963]]
model -- 0 new_R: 
[[ 2.44783055 -1.59545522]
 [-1.59571421  2.05858943]]
model -- 1 new_R: 
[[ 9.36370137 -7.67274031]
 [-7.67345803  7.79234342]]
new_Z: 
[[0.68974417 0.30523071]
 [0.68974417 0.30523071]]
-----------------------------------------------
-1239.3818769794066
model -- 0 new_Q: 
[[ 8.0367268  -2.18101765]
 [-2.1721766   7.1046831 ]]
model -- 1 new_Q: 
[[112.6823855   18.00439158]
 [ 18.0065185  125.58651094]]
model -- 0 new_R: 
[[ 2.44259405 -1.60365603]
 [-1.60390755  2.06419661]]
model -- 1 new_R: 
[[ 9.35625788 -7.70558919]
 [-7.7062865   7.83348299]]
new_Z: 
[[0.68959961 0.30537526]
 [0.68959961 0.30537526]]
-----------------------------------------------
-1239.390213461619
model -- 0 new_Q: 
[[ 8.03070275 -2.1879171 ]
 [-2.17919084  7.11013908]]
model -- 1 new_Q: 
[[112.64588328  18.06536567]
 [ 18

Filter model. Testing with learnt parameters from EMGPB2.

In [9]:
# em_transition_model_1 = LinearGaussianTimeInvariantTransitionModel(transition_matrix=new_models[0].A, covariance_matrix=transfer_Q(new_models[0].Q))
# em_transition_model_2 = LinearGaussianTimeInvariantTransitionModel(transition_matrix=new_models[1].A, covariance_matrix=transfer_Q(new_models[1].Q))

em_transition_model_1 = LinearGaussianTimeInvariantTransitionModel(transition_matrix=new_models[0].A, covariance_matrix=new_models[0].Q)
em_transition_model_2 = LinearGaussianTimeInvariantTransitionModel(transition_matrix=new_models[1].A, covariance_matrix=new_models[1].Q)
em_measurement_noise_1 = new_models[0].R
em_measurement_noise_2 = new_models[1].R

em_measurement_model_1 = LinearGaussian(ndim_state=2, mapping=[0, 1],
                                   noise_covar=em_measurement_noise_1)
em_measurement_model_2 = LinearGaussian(ndim_state=2, mapping=[0, 1],
                                   noise_covar=em_measurement_noise_2)

em_model_transition_matrix = np.array([[0.5, 0.5],
                                       [0.5, 0.5]])
em_predictor_1 = KalmanPredictor(em_transition_model_1)
em_predictor_2 = KalmanPredictor(em_transition_model_2)
em_imm_predictor = IMMPredictor([em_predictor_1, em_predictor_2], em_model_transition_matrix)

em_updater_1 = KalmanUpdater(em_measurement_model_1)
em_updater_2 = KalmanUpdater(em_measurement_model_2)
em_imm_updater = IMMUpdater([em_updater_1, em_updater_2], em_model_transition_matrix)

em_state_init = WeightedGaussianState(StateVector([[0], [0]]),
                                      CovarianceMatrix(np.diag([5.0, 5.0])),
                                      timestamp=timestamp_init,
                                      weight=0.5)

prior1 = copy(em_state_init)
prior = GaussianMixtureState([prior1, prior1])
track = Track([copy(prior)])
track_error = []

for i in range(1, len(detection_track)):
    measurement = detection_track[i]
    
    # State prediction
    prediction = em_imm_predictor.predict(track.state, timestamp=gt_time[i])
    # Measurement prediction
    meas_prediction = em_imm_updater.predict_measurement(prediction)
    # State update
    hyp = SingleHypothesis(prediction, measurement)
    prior = em_imm_updater.update(hyp)
    track.append(prior)
    track_error.append(em_measurement_model_1.function(track[i].state_vector, np.zeros((2,1))) - em_measurement_model_1.function(gt_track[i].state_vector, np.zeros((2,1))))
    
    '''
    fig, (ax1, ax2) = plt.subplots(2,1)
    # Generate plots
    ax1.cla()
    ax2.cla()
    # PLot true trajectory
    data = np.array([state.state_vector for state in gt_track])
    ax1.plot(data[:, 0], data[:, 2], 'b-')
    # PLot estimated trajectory
    data = np.array([state.state_vector for state in track.states])
    ax1.plot(data[:, 0], data[:, 2], 'r-')
    # Plot innovation covariance
    plot_cov_ellipse(meas_prediction.covar,
                     meas_prediction.mean, edgecolor='b',
                     facecolor='none', ax=ax1)
    # Plot estimated covariance
    plot_cov_ellipse(track.state.covar[[0, 2], :][:, [0, 2]],
                     track.state.mean[[0, 2], :], edgecolor='r',
                     facecolor='none', ax=ax1)
    # Visualise model weights
    ax2.bar([1,2], prior.weights.ravel())
    plt.pause(0.0001)
    '''

track_error = np.asarray(track_error)
track_error = np.squeeze(track_error)
rmse_tmp = []
for track_error_ in track_error:
    rmse_tmp.append(track_error_ @ track_error_.T)
rmse = np.sqrt(np.mean(np.array(rmse_tmp)))
print("RMSE: " + str(rmse))

RMSE: 1.219833105439567
